In [1]:
from random import choices, choice, shuffle
from typing import List
from datetime import date

from arr import Contract, ContractHeader, ContractLine

import pandas as pd
import numpy as np
from dateutil.relativedelta import relativedelta as rd

In [2]:
MIN_DATE = date(2020, 1, 1)
MAX_DATE = date(2026, 12, 31)
START_DATES = pd.date_range(MIN_DATE, MAX_DATE)
RENEWAL_CHANCE = .8
EXPANSION_CHANCE = .1
DOWNGRADE_CHANCE = .1

CUSTOMERS = pd.read_excel('data/saas_corp.xlsx','customer')
CUSTOMERS.set_index('key', inplace=True)

PRODUCTS = pd.read_excel('data/saas_corp.xlsx','product')
PRODUCTS.set_index('key', inplace=True)
PRODUCT_WEIGHTS = np.arange(0,1,1/len(PRODUCTS))[::-1]
PRODUCT_COUNT = len(PRODUCTS['product_name'].unique())

CONTRACTS = pd.read_excel('data/saas_corp.xlsx', 'contract')

In [3]:
def random_contract_range(
    contract_lengths: List[int] = [3, 6, 12, 24, 36],
    contract_weights: List[float] = [0.1, 0.05, 0.6, 0.2, 0.05],
) -> int:
    """Get a random contract length interval.

    Wrapper for:
    https://docs.python.org/3/library/random.html#random.choices

    Args:
        contract_lengths (List[int], optional): Possible contract lengths, in months.
            Defaults to [3, 6, 12, 24, 36].
        contract_weights (List[float], optional): Weights of the `contract_lengths`, arg.
            Defaults to [0.1, 0.05, 0.6, 0.2, 0.05].

    Returns:
        int: Length of contract in months.
    """
    return choices(contract_lengths, contract_weights)[0]

In [4]:
def initial_sale(customer: str):
    """Get customer start, end dates. Get customer items for first sale"""
    if customer in CONTRACTS["customer"].unique():
        print(f"{customer} already in contract list")
        return None

    start_date = choice(START_DATES)
    contract_range = random_contract_range()
    end_date = start_date + rd(months=contract_range)

    num_of_items = choices(range(1, PRODUCT_COUNT + 1), PRODUCT_WEIGHTS)[0]
    products = choices(PRODUCTS["product_name"].unique(), k=num_of_items)
    return customer, start_date, end_date, contract_range, num_of_items, products

In [5]:
for customer in CUSTOMERS['customer'].unique():
    initial_sale(customer)

Abatz already in contract list


In [6]:
a = Contract(
        1,
        ContractHeader(20_000, date(2024, 1, 1), date(2024, 12, 31)),
        [
            ContractLine(4_000, date(2024, 1, 1), date(2024, 12, 31), 1, True),
            ContractLine(4_000, date(2024, 1, 1), date(2024, 12, 31), 2, True),
            ContractLine(4_000, date(2024, 1, 1), date(2024, 12, 31), 3, True),
            ContractLine(4_000, date(2024, 1, 1), date(2024, 12, 31), 4, True),
            ContractLine(4_000, date(2024, 1, 1), date(2024, 12, 31), 5, False),
        ],
        'Abatz'
)

In [7]:
a.build_df()

,id,customer,header.amount,header.start_date,header.end_date,header.booking_date,line.amount,line.start_date,line.end_date,line.item_sku,line.renewable
0,1,Abatz,20000,2024-01-01,2024-12-31,2024-01-01,4000,2024-01-01,2024-12-31,1,True
1,1,Abatz,20000,2024-01-01,2024-12-31,2024-01-01,4000,2024-01-01,2024-12-31,2,True
2,1,Abatz,20000,2024-01-01,2024-12-31,2024-01-01,4000,2024-01-01,2024-12-31,3,True
3,1,Abatz,20000,2024-01-01,2024-12-31,2024-01-01,4000,2024-01-01,2024-12-31,4,True
4,1,Abatz,20000,2024-01-01,2024-12-31,2024-01-01,4000,2024-01-01,2024-12-31,5,False


In [ ]:
b = a.__dict__.copy()
b['header'] = a.header.__dict__.copy()
b['lines'] = [line.__dict__.copy() for line in a.lines]
b

In [ ]:
df = pd.json_normalize(b)
lines = pd.DataFrame.from_dict(df['lines'].iloc[0]).add_prefix('line.')
df.drop(columns=['lines'])
df.merge(lines, how='cross').drop(columns='lines')

In [ ]:
# product_weights = [.8, .6, .4, .2, .1, .05, .01, .001]

# add_item = True
# items = pd.DataFrame()
# potential_product = product
# item = choice(potential_product.index)
# potential_product.iloc[item-1:item]